In [1]:
pip install torch


In [2]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorboard

Note: you may need to restart the kernel to use updated packages.


In [15]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchvision.utils as vutils



In [5]:
device=torch.device("cpu")


In [6]:
# Create a summary writer object
writer = SummaryWriter(log_dir='/path/to/log/directory')


In [7]:
class PlantSeedClassifier(nn.Module):
    def __init__(self):
        super(PlantSeedClassifier,self).__init__()
        self.layer1 = nn.Linear(3*299*299,12)
        self.layer2 = nn.Linear(12,12)
        
        
    def forward(self,x):
        x = x.view(-1,3*299*299)
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
       

        x = self.layer2(x)
        return x
    

In [8]:
dt = "./plant-seedlings-final/test"
dtrain = "./plant-seedlings-final/train"


In [9]:

train_dataset = ImageFolder( dtrain,transform=transforms.ToTensor())



In [10]:
test_dataset = ImageFolder( dt,transform=transforms.ToTensor())


In [11]:
train_loader = DataLoader(train_dataset, batch_size=60, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=60, shuffle=False)


In [12]:
model = PlantSeedClassifier()
optimizer = optim.Adam(model.parameters(), lr=0.001)
writer.add_graph(model,input_to_model=torch.rand(1,268203))


In [14]:
for epoch in range(10):
    running_loss = 0.0
    for  i,data in enumerate(train_loader):
        inputs, labels= data
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = nn.functional.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
        div=100
        running_loss += loss.item()
        if [x % div for x in data] == 99:
            writer.add_scalar('training loss', running_loss / 100, (epoch * len(train_loader) + i) % len(train_dataset))
            running_loss = 0.0

    # Evaluate the model on the test set and write the results to TensorBoard
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    writer.add_scalar('test accuracy', accuracy, epoch)
    
print('Finished training')


Finished training


In [16]:
for i, data in enumerate(test_loader):
    images, _ = data
    grid = vutils.make_grid(images, normalize=True, scale_each=True)
    writer.add_image('images', grid, global_step=i)

# Close the SummaryWriter
writer.close()